### <div align="center">***RECOMENDADOR***</div>
***

In [1]:
import pandas as pd


In [2]:
import numpy as np
import pandas as pd
import scipy
import sys
import re
#nltk.download() 
import nltk

import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [3]:
#%pip install utilities

In [4]:

#!pip install unidecode
#%pip install utilities
sys.path.append(r"Utilities")
import Utilities 

ModuleNotFoundError: No module named 'Utilities'

# Limpieza dataset

In [ ]:
dataset=pd.read_csv('../data/df_comerios.csv')


In [ ]:
dataset=dataset.drop(["id",'image_url','is_closed','review_count','display_phone','location'], axis=1)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         958 non-null    object 
 1   rating       958 non-null    float64
 2   address      958 non-null    object 
 3   description  956 non-null    object 
 4   review       463 non-null    object 
 5   alias        958 non-null    object 
 6   latitude     958 non-null    float64
 7   longitude    958 non-null    float64
 8   price        620 non-null    object 
 9   distrito     958 non-null    object 
dtypes: float64(3), object(7)
memory usage: 75.0+ KB


In [ ]:
dataset.head(2)

,name,rating,address,description,review,alias,latitude,longitude,price,distrito
0,El Rincón Asturiano,1.0,"Calle de las Delicias, 26, 28045 Madrid, Spain",Spanish Asturian Tapas Bars,We are here for dinner again on two consecutiv...,el-rincón-asturiano-madrid-2,40.403985,-3.692258,€€,Arganzuela
1,Donde da la Vuelta el Viento,5.0,"Calle de Mesón de Paredes, 81, 28012 Madrid, S...",Tapas Bars Spanish Modern European,Great place with friendly staff. I came for ta...,donde-da-la-vuelta-el-viento-madrid,40.406190,-3.701441,€,Arganzuela


In [ ]:
dataset['combinacion'] = dataset[['description', 'review', 'alias','distrito']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [ ]:
dataset=dataset.drop(['description', 'review', 'alias','distrito'], axis=1)

In [ ]:
dataset.head(2)

,name,rating,address,latitude,longitude,price,combinacion
0,El Rincón Asturiano,1.0,"Calle de las Delicias, 26, 28045 Madrid, Spain",40.403985,-3.692258,€€,Spanish Asturian Tapas Bars We are here for d...
1,Donde da la Vuelta el Viento,5.0,"Calle de Mesón de Paredes, 81, 28012 Madrid, S...",40.406190,-3.701441,€,Tapas Bars Spanish Modern European Great plac...


# Limpieza de columna analisis

Lo primero separamos el texto en palabras


In [ ]:
#Tokenization
dataset['Descripcion_tok'] = dataset['combinacion'].apply(lambda x: Utilities.tokenizacion(x)) 
dataset['Descripcion_tok'].head(10)

NameError: name 'Utilities' is not defined

Ahora eliminamos palabras que aportan poco significado: articulos, preposiciones.

In [ ]:
dataset['Descripcion_tok'] = dataset['Descripcion_tok'].apply(lambda x:Utilities.removeStopwords(x)) 
dataset['Descripcion_tok'].head(10)

Ahora eliminamos signos de puntuación 

In [ ]:

dataset['Descripcion_tok'] = dataset['Descripcion_tok'].apply(lambda x:Utilities.removePunctuation(x)) 
dataset['Descripcion_tok'].head(10) 

Ahora unimos todas las palabras, lo convertimos a string.

In [ ]:

dataset['Descripcion_clean'] = dataset['Descripcion_tok'].apply(lambda x:Utilities.arrayToString(x))
dataset['Descripcion_clean'].head(10)

# Analisis importancia palabras

In [ ]:
# Contamos cada palabra y creamos con esta cuenta una matriz

vectorizer = CountVectorizer(encoding='iso-8859-1')  
MatrizFrecuencias = vectorizer.fit_transform(dataset['Descripcion_clean'])
MatrizFrecuencias

NameError: name 'CountVectorizer' is not defined

In [ ]:
# Visualizacion matriz de Frecuencias

X = pd.DataFrame(data=MatrizFrecuencias.toarray(), index= dataset['name'].values,
                columns=vectorizer.get_feature_names_out())
X

In [ ]:
# Porcentaje de ceros por palabras, nos hacemos una idea de que palabras se repiten mucho y tienen menos importancia para el análisis

PorecntajeCerosPorPalabra = X.apply(lambda x: 100*len(np.where(x ==0)[0])/len(x), axis=0).sort_values(ascending=True)
PorecntajeCerosPorPalabra.head(10)

Calculamos el término frecuencia inversa de documento, para calcular la proporción de las veces que aparece cada palabra en relación con el resto de palabras ( matriz de correlación).


In [ ]:

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(MatrizFrecuencias)
tfidf

Ahora que ya tenemos la similitud entre las palabras, podemos clacular la similitud entre los restaurantes, en funcion de las veces que aparecen estas palabras en cada uno de ellos.

In [ ]:
tdm = tfidf.transpose()
dtm = tfidf
Simil = dtm.dot(tdm) #TRANSPONES LA MATRIZ PARA MULTIPLICARLA POR SI MISMA (MATRIZ AL CUADRADO)

Visualizamos el resultado de la matriz de similitud


In [ ]:
SimilDF = pd.DataFrame(data = Simil.toarray(), index=dataset['name'].values,columns=dataset['name'].values)
SimilDF

In [ ]:
Top = 10
Num_restaurante = 55
print('restaurante:',SimilDF.columns[Num_restaurante])
RecomendacionItemItem = SimilDF.iloc[(-SimilDF.iloc[:, Num_restaurante]).argsort()[1:(Top+1)].values, Num_restaurante] 
print ('\n Los restaurantes más similares son:')
RecomendacionItemItem